In [1]:
from __future__ import annotations
import xarray as xr
import fsspec
from pathlib import Path
from dask.diagnostics import ProgressBar
import pandas as pd
import numpy as np
import scipy.spatial
from typing import Iterable, List, Optional, Sequence, Union, Dict, Tuple
import os
import metview as mv
import time

In [15]:
def print_era5_vars(ds):
    rows = []
    for name, da in ds.data_vars.items():
        long_name = da.attrs.get("long_name", "")
        units = da.attrs.get("units", "")
        rows.append((name, long_name, units))

    # column widths
    name_w = max(len(r[0]) for r in rows)
    lname_w = max(len(r[1]) for r in rows)
    units_w = max(len(r[2]) for r in rows)

    header = (
        f"{'VAR':<{name_w}} | "
        f"{'LONG NAME':<{lname_w}} | "
        f"{'UNITS':<{units_w}}"
    )
    sep = "-" * len(header)

    print(header)
    print(sep)
    for name, long_name, units in rows:
        print(
            f"{name:<{name_w}} | "
            f"{long_name:<{lname_w}} | "
            f"{units:<{units_w}}"
        )


In [12]:
fs = fsspec.filesystem('gs', token='anon')

In [34]:
ds = xr.open_zarr(
    fs.get_mapper('gs://gcp-public-data-arco-era5/co/single-level-reanalysis.zarr-v2'),
    consolidated=True,
)
print_era5_vars(ds)


/tmp/ipykernel_1059544/879621314.py:1: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  ds = xr.open_zarr(


VAR     | LONG NAME                                                  | UNITS         
-------------------------------------------------------------------------------------
cape    | Convective available potential energy                      | J kg**-1      
d2m     | 2 metre dewpoint temperature                               | K             
hcc     | High cloud cover                                           | (0 - 1)       
istl1   | Ice temperature layer 1                                    | K             
istl2   | Ice temperature layer 2                                    | K             
istl3   | Ice temperature layer 3                                    | K             
istl4   | Ice temperature layer 4                                    | K             
lcc     | Low cloud cover                                            | (0 - 1)       
mcc     | Medium cloud cover                                         | (0 - 1)       
msl     | Mean sea level pressure                     

In [32]:
ds

<xarray.Dataset> Size: 1PB
Dimensions:     (time: 1323648, hybrid: 137, values: 410240)
Coordinates:
  * time        (time) datetime64[ns] 11MB 1900-01-01 ... 2050-12-31T23:00:00
  * hybrid      (hybrid) float64 1kB 1.0 2.0 3.0 4.0 ... 134.0 135.0 136.0 137.0
    step        timedelta64[ns] 8B ...
    valid_time  (time) datetime64[ns] 11MB dask.array<chunksize=(1323648,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    d           (time, hybrid, values) float32 298TB dask.array<chunksize=(1, 1, 410240), meta=np.ndarray>
    t           (time, hybrid, values) float32 298TB dask.array<chunksize=(1, 1, 410240), meta=np.ndarray>
    vo          (time, hybrid, values) float32 298TB dask.array<chunksize=(1, 1, 410240), meta=np.ndarray>
    w           (time, hybrid, values) float32 298TB dask.array<chunksize=(1, 1, 410240), meta=np.ndarray>
Attributes: (12/14)
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    European Centre for Medium-Range Weather Forec...
    GRIB_edition:              2
    GRIB_subCentre:            0
    history:                   2022-10-09T17:47 GRIB to CDM+CF via cfgrib-0.9...
    ...                        ...
    pangeo-forge:recipe_hash:  ed5fbf407898def1e27d4e8965c2cdaa01b7ea8f2ac2e1...
    pangeo-forge:version:      0.9.1
    valid_time_start:          1940-01-01
    last_updated:              2026-01-16 01:58:25.664990+00:00
    valid_time_stop:           2025-10-31
    valid_time_stop_era5t:     2026-01-10

In [19]:
ds = xr.open_zarr(
    fs.get_mapper('gs://gcp-public-data-arco-era5/co/model-level-wind.zarr-v2'),
    consolidated=True,
)
ds


/tmp/ipykernel_1059544/421711231.py:1: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  ds = xr.open_zarr(


<xarray.Dataset> Size: 1PB
Dimensions:     (time: 1323648, hybrid: 137, values: 410240)
Coordinates:
  * time        (time) datetime64[ns] 11MB 1900-01-01 ... 2050-12-31T23:00:00
  * hybrid      (hybrid) float64 1kB 1.0 2.0 3.0 4.0 ... 134.0 135.0 136.0 137.0
    step        timedelta64[ns] 8B ...
    valid_time  (time) datetime64[ns] 11MB dask.array<chunksize=(1323648,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    d           (time, hybrid, values) float32 298TB dask.array<chunksize=(1, 1, 410240), meta=np.ndarray>
    t           (time, hybrid, values) float32 298TB dask.array<chunksize=(1, 1, 410240), meta=np.ndarray>
    vo          (time, hybrid, values) float32 298TB dask.array<chunksize=(1, 1, 410240), meta=np.ndarray>
    w           (time, hybrid, values) float32 298TB dask.array<chunksize=(1, 1, 410240), meta=np.ndarray>
Attributes: (12/14)
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    European Centre for Medium-Range Weather Forec...
    GRIB_edition:              2
    GRIB_subCentre:            0
    history:                   2022-10-09T17:47 GRIB to CDM+CF via cfgrib-0.9...
    ...                        ...
    pangeo-forge:recipe_hash:  ed5fbf407898def1e27d4e8965c2cdaa01b7ea8f2ac2e1...
    pangeo-forge:version:      0.9.1
    valid_time_start:          1940-01-01
    last_updated:              2026-01-16 01:58:25.664990+00:00
    valid_time_stop:           2025-10-31
    valid_time_stop_era5t:     2026-01-10

In [20]:
ds["hybrid"].values

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
       122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
       133., 134., 135., 136., 137.])

In [22]:
print([*ds.coords])
print([*ds.data_vars])


['hybrid', 'step', 'time', 'valid_time']
['d', 't', 'vo', 'w']


In [23]:
ds = xr.open_zarr(
    fs.get_mapper('gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3'),
    consolidated=True,
)

In [28]:
ds['temperature'].sel(level=250)

<xarray.DataArray 'temperature' (time: 1323648, latitude: 721, longitude: 1440)> Size: 5TB
dask.array<getitem, shape=(1323648, 721, 1440), dtype=float32, chunksize=(1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 11MB 1900-01-01 ... 2050-12-31T23:00:00
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    level      int64 8B 250
Attributes:
    long_name:      Temperature
    short_name:     t
    standard_name:  air_temperature
    units:          K

In [30]:
for t in pd.date_range("2020-01-01T00:00:00", periods=24, freq="H"):
    one_hour = ds["temperature"].sel(level=250, time=t)
    arr = one_hour.values                  # (542080,)
    print(t, arr)

/tmp/ipykernel_1059544/1803027966.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for t in pd.date_range("2020-01-01T00:00:00", periods=24, freq="H"):


2020-01-01 00:00:00 [[208.00505 208.00505 208.00505 ... 208.00505 208.00505 208.00505]
 [207.94408 207.94408 207.9452  ... 207.9435  207.9435  207.94408]
 [207.78506 207.78621 207.78679 ... 207.78223 207.78336 207.78394]
 ...
 [227.25282 227.25397 227.2551  ... 227.25111 227.25168 227.25282]
 [227.33888 227.33888 227.34001 ... 227.33774 227.33774 227.33888]
 [227.45229 227.45229 227.45229 ... 227.45229 227.45229 227.45229]]
2020-01-01 01:00:00 [[207.91045 207.91045 207.91045 ... 207.91045 207.91045 207.91045]
 [207.80559 207.80559 207.80559 ... 207.80388 207.80502 207.80502]
 [207.68364 207.68477 207.68535 ... 207.68193 207.68193 207.6825 ]
 ...
 [227.30013 227.30013 227.30013 ... 227.29785 227.29898 227.29898]
 [227.41695 227.41809 227.41809 ... 227.4158  227.41695 227.41695]
 [227.5315  227.5315  227.5315  ... 227.5315  227.5315  227.5315 ]]
2020-01-01 02:00:00 [[207.85403 207.85403 207.85403 ... 207.85403 207.85403 207.85403]
 [207.77766 207.77881 207.77881 ... 207.77766 207.77766 2

In [31]:
def attribute_fix(ds: xr.Dataset) -> xr.Dataset:
    """Fixes ecCodes shortName ambiguity by removing overloaded GRIB_* attrs (as in your notebook)."""
    ds = ds.copy()
    for var in ds.data_vars:
        attrs = dict(ds[var].attrs)
        attrs.pop("GRIB_cfName", None)
        attrs.pop("GRIB_cfVarName", None)
        attrs.pop("GRIB_shortName", None)
        ds[var].attrs = attrs
    return ds


def _slice_one_hour(ds: xr.Dataset, time) -> xr.Dataset:
    """
    Select exactly one timestep from ds at the given 'time'.
    Raises ValueError if 0 or >1 timesteps result.
    """
    if "time" not in ds.coords and "time" not in ds.dims:
        raise ValueError("Expected dataset to have a 'time' coordinate/dimension.")

    try:
        sel = ds.sel(time=time)
    except Exception as e:
        raise ValueError(f"Failed to select time={time!r} from dataset. Original error: {e}") from e

    # Normalize to having an explicit time dimension for checking
    if "time" in sel.dims:
        n = int(sel.sizes["time"])
        if n != 1:
            raise ValueError(f"Expected exactly one timestep for time={time!r}, got {n}.")
        sel = sel.isel(time=0)
    else:
        # time may have been squeezed out into a scalar coord already; that's fine
        pass

    # Materialize (Metview needs concrete arrays for conversion)
    return sel.compute()


def _ds_to_fieldset(ds_hour: xr.Dataset) -> mv.Fieldset:
    """xarray Dataset (single hour) -> Metview Fieldset, with attribute_fix + squeeze."""
    ds_hour = attribute_fix(ds_hour).squeeze()
    return mv.dataset_to_fieldset(ds_hour)


def _maybe_to_gaussian_grid(fs: mv.Fieldset, gaussian_grid: Optional[str]) -> mv.Fieldset:
    """Optionally normalize to a Gaussian grid like 'N320' (as in your notebook)."""
    if gaussian_grid is None:
        return fs
    return mv.read(data=fs, grid=gaussian_grid)


def _select_shortname(fs: mv.Fieldset, shortname: str) -> mv.Fieldset:
    sel = fs.select(shortName=shortname)
    if len(sel) == 0:
        raise KeyError(f"Variable shortName={shortname!r} not found in input Fieldset.")
    return sel


def _type_of_level(fs: mv.Fieldset) -> str:
    """
    Return the (unique) GRIB typeOfLevel for this Fieldset.
    If multiple are found, returns the first but includes a warning-like detail in the error paths.
    """
    # mv.grib_get returns number/str/list depending on input fieldset size
    vals = mv.grib_get(fs, "typeOfLevel")
    if isinstance(vals, str):
        return vals
    if isinstance(vals, (list, tuple)):
        uniq = [v for v in dict.fromkeys(vals)]  # preserve order
        return str(uniq[0]) if uniq else ""
    return str(vals)


def _ensure_model_level(fs: mv.Fieldset, shortname: str) -> None:
    tol = _type_of_level(fs)
    if tol != "hybrid":
        raise ValueError(
            f"Variable {shortname!r} is not on model (hybrid) levels (typeOfLevel={tol!r}); "
            "cannot interpolate to pressure levels."
        )


def _ensure_not_model_level(fs: mv.Fieldset, shortname: str) -> None:
    tol = _type_of_level(fs)
    if tol == "hybrid":
        raise ValueError(
            f"Variable {shortname!r} is on model (hybrid) levels; you must provide pressure_hpa."
        )


def _regrid_latlon(fs: mv.Fieldset, out_res_deg: float) -> mv.Fieldset:
    """Regrid to a regular lat/lon grid of resolution out_res_deg."""
    return mv.read(data=fs, grid=[float(out_res_deg), float(out_res_deg)])


def _fieldset_to_xr(fs: mv.Fieldset) -> xr.Dataset:
    """Convert Fieldset -> xarray.Dataset via cfgrib."""
    return fs.to_dataset()


def _normalize_pressures(pressure_hpa: Optional[Sequence[int]]) -> Optional[List[int]]:
    if pressure_hpa is None:
        return None
    if isinstance(pressure_hpa, (list, tuple, np.ndarray)):
        pressures = [int(p) for p in pressure_hpa]
        if len(pressures) == 0:
            raise ValueError("pressure_hpa was provided but empty. Use None or a non-empty list like [250, 500].")
        return pressures
    raise ValueError("pressure_hpa must be None or a list/tuple/ndarray of ints like [250, 500].")


# ----------------------------
# Main entrypoint
# ----------------------------

# def metview_extract_latlon_at_time(
#     input_ds: xr.Dataset,
#     vars: Sequence[str],
#     time,
#     *,
#     surface_ds: Optional[xr.Dataset] = None,
#     pressure_hpa: Optional[Sequence[int]] = None,  # None or [250, 500, ...]
#     out_res_deg: float = 0.25,
#     gaussian_grid: Optional[str] = "N320",
#     to_xarray: bool = True,
# ) -> Union[xr.Dataset, mv.Fieldset]:
#     """
#     Given an xarray Dataset (surface OR model-level), extract variables at a single hour,
#     (optionally) interpolate model-level variables to given pressure levels, regrid to regular lat/lon,
#     and optionally return as xarray.Dataset.

#     Contract / error modes:
#       - If pressure_hpa is provided:
#           * surface_ds must be provided and must contain lnsp at the same hour
#           * each requested var must be model-level (typeOfLevel='hybrid') or ValueError
#       - If pressure_hpa is None:
#           * each requested var must NOT be model-level 'hybrid' (otherwise ValueError)
#       - If time selection yields not exactly one timestep: ValueError
#       - If a requested var is missing: KeyError
#     """
#     pressures = _normalize_pressures(pressure_hpa)
#     if not vars:
#         raise ValueError("vars must be a non-empty list of Metview shortNames, e.g. ['u','v'] or ['t'].")

#     # 1) Slice input dataset to exactly one hour and materialize
#     input_hour = _slice_one_hour(input_ds, time)

#     # 2) If pressure interpolation requested, slice surface and require lnsp
#     surface_hour = None
#     if pressures is not None:
#         if surface_ds is None:
#             raise ValueError("pressure_hpa was provided, but surface_ds is required to supply lnsp.")
#         surface_hour = _slice_one_hour(surface_ds, time)

#     # 3) Convert to Metview Fieldsets
#     input_fs = _ds_to_fieldset(input_hour)
#     input_fs = _maybe_to_gaussian_grid(input_fs, gaussian_grid)

#     surface_fs = None
#     lnsp_fs = None
#     if pressures is not None:
#         surface_fs = _ds_to_fieldset(surface_hour)  # type: ignore[arg-type]
#         surface_fs = _maybe_to_gaussian_grid(surface_fs, gaussian_grid)
#         lnsp_fs = _select_shortname(surface_fs, "lnsp")

#     # 4) For each variable: select, validate, (optional) interpolate, then regrid
#     out_fieldsets: List[mv.Fieldset] = []

#     for shortname in vars:
#         var_fs = _select_shortname(input_fs, shortname)

#         if pressures is None:
#             # Regrid-only mode: model-level variables are user error
#             _ensure_not_model_level(var_fs, shortname)
#             var_ll = _regrid_latlon(var_fs, out_res_deg)
#             out_fieldsets.append(var_ll)
#         else:
#             # Pressure mode: variables must be hybrid (model-level)
#             _ensure_model_level(var_fs, shortname)
#             # Interpolate to pressure(s) in hPa
#             var_pl = mv.mvl_ml2hPa(lnsp_fs, var_fs, pressures)  # type: ignore[arg-type]
#             var_ll = _regrid_latlon(var_pl, out_res_deg)
#             out_fieldsets.append(var_ll)

#     # 5) Merge outputs
#     merged = out_fieldsets[0]
#     for fs in out_fieldsets[1:]:
#         merged = mv.merge(merged, fs)

#     if not to_xarray:
#         return merged

#     # 6) Convert to xarray
#     xr_out = _fieldset_to_xr(merged)

#     # Optional: keep only requested vars if cfgrib pulled extras (rare but can happen)
#     # We'll be conservative and filter by presence.
#     keep = [v for v in vars if v in xr_out.data_vars]
#     if keep:
#         xr_out = xr_out[keep]

#     return xr_out

def metview_extract_latlon_at_time(
    input_ds: xr.Dataset,
    vars: Sequence[str],
    time_sel,
    *,
    surface_ds: Optional[xr.Dataset] = None,
    pressure_hpa: Optional[Sequence[int]] = None,
    out_res_deg: float = 0.25,
    gaussian_grid: Optional[str] = "N320",
    to_xarray: bool = True,
) -> Union[xr.Dataset, mv.Fieldset]:

    print("\n--- metview_extract_latlon_at_time ---", flush=True)

    t0 = time.time()
    pressures = _normalize_pressures(pressure_hpa)
    print(f"normalize_pressures: {time.time() - t0:.3f}s", flush=True)

    if not vars:
        raise ValueError("vars must be non-empty")

    t0 = time.time()
    input_hour = _slice_one_hour(input_ds, time_sel)
    print(f"slice input_ds: {time.time() - t0:.3f}s", flush=True)

    surface_hour = None
    if pressures is not None:
        if surface_ds is None:
            raise ValueError("surface_ds required for pressure interpolation")

        t0 = time.time()
        surface_hour = _slice_one_hour(surface_ds, time_sel)
        print(f"slice surface_ds: {time.time() - t0:.3f}s", flush=True)

    t0 = time.time()
    input_fs = _ds_to_fieldset(input_hour)
    print(f"ds_to_fieldset(input): {time.time() - t0:.3f}s", flush=True)

    t0 = time.time()
    input_fs = _maybe_to_gaussian_grid(input_fs, gaussian_grid)
    print(f"maybe_to_gaussian_grid(input): {time.time() - t0:.3f}s", flush=True)

    surface_fs = None
    lnsp_fs = None
    if pressures is not None:
        t0 = time.time()
        surface_fs = _ds_to_fieldset(surface_hour)
        print(f"ds_to_fieldset(surface): {time.time() - t0:.3f}s", flush=True)

        t0 = time.time()
        surface_fs = _maybe_to_gaussian_grid(surface_fs, gaussian_grid)
        print(f"maybe_to_gaussian_grid(surface): {time.time() - t0:.3f}s", flush=True)

        t0 = time.time()
        lnsp_fs = _select_shortname(surface_fs, "lnsp")
        print(f"select lnsp: {time.time() - t0:.3f}s", flush=True)

    out_fieldsets: List[mv.Fieldset] = []

    for shortname in vars:
        t0 = time.time()
        var_fs = _select_shortname(input_fs, shortname)
        print(f"select {shortname}: {time.time() - t0:.3f}s", flush=True)

        if pressures is None:
            t0 = time.time()
            _ensure_not_model_level(var_fs, shortname)
            print(f"validate not model-level ({shortname}): {time.time() - t0:.3f}s", flush=True)

            t0 = time.time()
            var_ll = _regrid_latlon(var_fs, out_res_deg)
            print(f"regrid latlon ({shortname}): {time.time() - t0:.3f}s", flush=True)

            out_fieldsets.append(var_ll)

        else:
            t0 = time.time()
            _ensure_model_level(var_fs, shortname)
            print(f"validate model-level ({shortname}): {time.time() - t0:.3f}s", flush=True)

            t0 = time.time()
            var_pl = mv.mvl_ml2hPa(lnsp_fs, var_fs, pressures)
            print(f"ml2hPa interpolate ({shortname}): {time.time() - t0:.3f}s", flush=True)

            t0 = time.time()
            var_ll = _regrid_latlon(var_pl, out_res_deg)
            print(f"regrid latlon ({shortname}): {time.time() - t0:.3f}s", flush=True)

            out_fieldsets.append(var_ll)

    t0 = time.time()
    merged = out_fieldsets[0]
    for fs in out_fieldsets[1:]:
        merged = mv.merge(merged, fs)
    print(f"merge outputs: {time.time() - t0:.3f}s", flush=True)

    if not to_xarray:
        return merged

    t0 = time.time()
    xr_out = _fieldset_to_xr(merged)
    print(f"fieldset_to_xr: {time.time() - t0:.3f}s", flush=True)

    keep = [v for v in vars if v in xr_out.data_vars]
    if keep:
        xr_out = xr_out[keep]

    print(f"TOTAL: done", flush=True)
    return xr_out



In [32]:
fs = fsspec.filesystem('gs', token='anon')
fs.ls('gs://gcp-public-data-arco-era5/co')


['gcp-public-data-arco-era5/co/model-level-moisture.zarr',
 'gcp-public-data-arco-era5/co/model-level-moisture.zarr-v2',
 'gcp-public-data-arco-era5/co/model-level-wind.zarr',
 'gcp-public-data-arco-era5/co/model-level-wind.zarr-v2',
 'gcp-public-data-arco-era5/co/single-level-forecast.zarr',
 'gcp-public-data-arco-era5/co/single-level-forecast.zarr-v2',
 'gcp-public-data-arco-era5/co/single-level-reanalysis.zarr',
 'gcp-public-data-arco-era5/co/single-level-reanalysis.zarr-v2',
 'gcp-public-data-arco-era5/co/single-level-surface.zarr',
 'gcp-public-data-arco-era5/co/single-level-surface.zarr-v2']

In [33]:
ml_surface_ds = xr.open_zarr(
    fs.get_mapper('gs://gcp-public-data-arco-era5/co/single-level-reanalysis.zarr'),
    consolidated=True,
)
ml_wind_ds = xr.open_zarr(
    fs.get_mapper('gs://gcp-public-data-arco-era5/co/model-level-wind.zarr-v2'),
    consolidated=True,
)

/tmp/ipykernel_3846/4141780327.py:1: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  ml_surface_ds = xr.open_zarr(
/tmp/ipykernel_3846/4141780327.py:5: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding i

In [1]:
out = metview_extract_latlon_at_time(
    input_ds=ml_wind_ds[['d', 't']],
    vars=["d", "t"],
    time_sel="2017-10-09T06",
    surface_ds=ml_surface_ds,
    pressure_hpa=[250, 500],
    out_res_deg=0.25,
)



KeyboardInterrupt



In [ ]:
ml_wind_ds[['d', 't']].sel(time="2020-01-01T00:00:00").compute()

In [10]:
_slice_one_hour(ml_wind_ds, "2020-01-01T00:00:00")

KeyboardInterrupt: 

In [13]:
ar = xr.open_zarr(
    fs.get_mapper('gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3'),
    consolidated=True,
    group="u_component_of_wind"
)

AttributeError: 'Array' object has no attribute 'synchronizer'

In [2]:
import xarray as xr
import fsspec
import zarr
import dask.array as da

fs = fsspec.filesystem("gs", token="anon")

# Dataset root (for coords)
root = "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3"
store_root = fs.get_mapper(root)

# Array paths (for fast open of only u/v metadata)
rootU = root + "/u_component_of_wind"
rootV = root + "/v_component_of_wind"
storeU = fs.get_mapper(rootU)
storeV = fs.get_mapper(rootV)

# Open only the array metadata you care about
u_z = zarr.open_array(storeU, mode="r")
v_z = zarr.open_array(storeV, mode="r")

# Lazy dask arrays (no data read yet)
u_d = da.from_zarr(u_z)
v_d = da.from_zarr(v_z)

# Dimension names written by xarray into zarr attrs
dims = tuple(u_z.attrs["_ARRAY_DIMENSIONS"])

# Load coordinate arrays from the dataset root (small; ok to read)
# (Only include coords that actually exist as arrays at the root.)
root_group = zarr.open_group(store_root, mode="r")
coords = {d: root_group[d][:] for d in dims if d in root_group}

u = xr.DataArray(u_d, dims=dims, coords=coords, name="u_component_of_wind")
v = xr.DataArray(v_d, dims=dims, coords=coords, name="v_component_of_wind")

uv = xr.Dataset({"u_component_of_wind": u, "v_component_of_wind": v})


In [3]:
import dask.array as da
import numpy as np

n = uv.sizes["time"]
BASE = np.datetime64("1900-01-01T00:00:00")

time_dt = BASE + da.arange(n, chunks=100_000).astype("timedelta64[h]")
uv = uv.assign_coords(time=("time", time_dt))


In [4]:
with ProgressBar():
    uv.sel(level=250, time="2020-01-01T00:00:00").compute()

[########################################] | 100% Completed | 39.51 s


In [23]:
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler, ProgressBar

x = uv["u_component_of_wind"].sel(time="2020-01-01T00:00:00", level=250)

with Profiler() as prof, ResourceProfiler(dt=0.2) as rprof, CacheProfiler() as cprof, ProgressBar():
    out = x.compute()

print("tasks:", len(prof.results))
print("peak mem (MB):", max(r.mem for r in rprof.results))
print("peak cpu (%):", max(r.cpu for r in rprof.results))


[########################################] | 100% Completed | 15.28 s
tasks: 2
peak mem (MB): 2475.536384
peak cpu (%): 109.7


In [33]:
import numpy as np
import pandas as pd

BASE = np.datetime64("1900-01-01T00:00:00")

def dt_to_time_index(dt) -> int:
    dt64 = np.datetime64(pd.Timestamp(dt).to_datetime64())
    return int((dt64 - BASE) / np.timedelta64(1, "h"))

i = dt_to_time_index("2020-01-01T00:00:00")
arr = u_z[i, :, :, :]
i = dt_to_time_index("2020-01-01T00:00:00")

u_hour = u_z[i, :, :, :]        # shape (37,721,1440)
v_hour = v_z[i, :, :, :]

level_vals = root_group["level"][:]   # pressure levels in hPa
lvl_idx = int(np.where(level_vals == 250)[0][0])

u_250 = u_hour[lvl_idx]
v_250 = v_hour[lvl_idx]


In [34]:
import time

i = dt_to_time_index("2020-01-01T00:00:00")

t0 = time.time()
u_hour = u_z[i, :, :, :]
print("u read:", time.time() - t0)

t0 = time.time()
v_hour = v_z[i, :, :, :]
print("v read:", time.time() - t0)


u read: 21.821593284606934
v read: 8.447362184524536


In [36]:
import time
i = dt_to_time_index("2020-01-01T00:00:00")

t0 = time.time(); _ = u_z[i, :, :, :]; print("u first:", time.time()-t0)
t0 = time.time(); _ = u_z[i, :, :, :]; print("u second:", time.time()-t0)


u first: 37.61649751663208
u second: 34.41849184036255


In [38]:
fullAR = xr.open_zarr(
    fs.get_mapper('gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3'),
    consolidated=True,
)

In [40]:
fullAR.sel(time="2020-01-01T00:00:00", level=250)

<xarray.Dataset> Size: 1GB
Dimensions:                                                          (
                                                                      latitude: 721,
                                                                      longitude: 1440)
Coordinates:
  * latitude                                                         (latitude) float32 3kB ...
  * longitude                                                        (longitude) float32 6kB ...
    level                                                            int64 8B ...
    time                                                             datetime64[ns] 8B ...
Data variables: (12/273)
    100m_u_component_of_wind                                         (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    100m_v_component_of_wind                                         (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    10m_u_component_of_neutral_wind                                  (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    10m_u_component_of_wind                                          (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    10m_v_component_of_neutral_wind                                  (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                                          (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    ...                                                               ...
    wave_spectral_directional_width_for_swell                        (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    wave_spectral_directional_width_for_wind_waves                   (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    wave_spectral_kurtosis                                           (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    wave_spectral_peakedness                                         (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    wave_spectral_skewness                                           (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    zero_degree_level                                                (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
Attributes:
    last_updated:           2026-01-17 02:00:10.664148+00:00
    valid_time_start:       1940-01-01
    valid_time_stop:        2025-10-31
    valid_time_stop_era5t:  2026-01-11

In [1]:
import xarray as xr
import fsspec
import zarr
import dask.array as da

fs = fsspec.filesystem("gs", token="anon")

# Dataset root (for coords)
root = "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3"
store_root = fs.get_mapper(root)

# Array paths (for fast open of only u/v metadata)
rootU = root + "/u_component_of_wind"
rootV = root + "/v_component_of_wind"
storeU = fs.get_mapper(rootU)
storeV = fs.get_mapper(rootV)

# Open only the array metadata you care about
u_z = zarr.open_array(storeU, mode="r")
v_z = zarr.open_array(storeV, mode="r")

# Lazy dask arrays (no data read yet)
u_d = da.from_zarr(u_z)
v_d = da.from_zarr(v_z)

# Dimension names written by xarray into zarr attrs
dims = tuple(u_z.attrs["_ARRAY_DIMENSIONS"])

# Load coordinate arrays from the dataset root (small; ok to read)
# (Only include coords that actually exist as arrays at the root.)
root_group = zarr.open_group(store_root, mode="r")
coords = {d: root_group[d][:] for d in dims if d in root_group}

u = xr.DataArray(u_d, dims=dims, coords=coords, name="u_component_of_wind")
v = xr.DataArray(v_d, dims=dims, coords=coords, name="v_component_of_wind")

uv = xr.Dataset({"u_component_of_wind": u, "v_component_of_wind": v})

import dask.array as da
import numpy as np

n = uv.sizes["time"]
BASE = np.datetime64("1900-01-01T00:00:00")

time_dt = BASE + da.arange(n, chunks=100_000).astype("timedelta64[h]")
uv = uv.assign_coords(time=("time", time_dt))

data = uv.sel(level=250, time="2020-01-01T00:00:00").compute()

